In [4]:
from torch.utils.data._utils.collate import default_collate
from torchvision import transforms
import torch
import torchvision.transforms.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import os
from PIL import Image
import xml.etree.ElementTree as ET
import sys
from tqdm import tqdm
import tempfile
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
from sklearn.metrics import classification_report
import torchvision.models.detection as detection
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, FasterRCNN_ResNet50_FPN_Weights
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import json
from torchvision.ops import nms
import logging
import numpy as np


In [5]:
#importing needed packages and modules 
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms
import torchvision.transforms.functional as F
from PIL import Image
import os
import xml.etree.ElementTree as ET
import sys
from torch.utils.data._utils.collate import default_collate

"""
collate function :
this function will collate data samples into batches 
images and boxes which are the annotations are seperated from each batch 
we will be used default collate to collate the images which is a function provided by pytorch 
targets are returned as a list of dictionaries 
"""
def collate_fn(batch):
    batch_images = [item[0] for item in batch]
    batch_targets = [item[1] for item in batch]
    batch_images_collated = default_collate(batch_images)
    return batch_images_collated, batch_targets

#category_id_mapping is a dictionary that maps the IDs to different acne type 
category_id_mapping = {
    1: 'Acne',
    2: 'Blackhead',
    3: 'Conglobata',
    4: 'Crystanlline',
    5: 'Cystic',
    6: 'Flat_wart',
    7: 'Folliculitis',
    8: 'Keloid',
    9: 'Milium',
    10: 'Papular',
    11: 'Purulent',
    12: 'Scars',
    13: 'Sebo-crystan-conglo',
    14: 'Syringoma',
    15: 'Whitehead',
    16: 'null'
}

# this is the reverse of category_id_mapping as it provides category of the acne to its integer which represents the ID 
name_to_id = {v: k for k, v in category_id_mapping.items()}

'''
next we have created different classes that implement different transofrmations to the images while preserving the bounding boxes
'''
#ToTensorWithTarget this class will convert PIL images to pytorch tensor and the bounding boxes are not changes 
class ToTensorWithTarget:
    def __call__(self, image, target):
        return transforms.ToTensor()(image), target
    
#ColorJitterWithTarget will apply different color editing such as brightness, contrast, saturation and hue to the images while also not making any changes to the bounding bounding boxes 
class ColorJitterWithTarget:
    def __init__(self, brightness=0, contrast=0, saturation=0, hue=0):
        self.color_jitter = transforms.ColorJitter(brightness=brightness, contrast=contrast, saturation=saturation, hue=hue)

    def __call__(self, image, target):
        image = self.color_jitter(image)
        return image, target
    
#RandomHorizontalFlipWithBBox  will flip the images horizantally while also adjusting the bounding boxes accornding to the flipes that were made to the images 
class RandomHorizontalFlipWithBBox(transforms.RandomHorizontalFlip):
    def __call__(self, image, target):
        if torch.rand(1) < self.p:
            image = F.hflip(image)
            if 'boxes' in target and target['boxes'].numel() > 0:
                width = image.shape[-1]
                xmin = width - target['boxes'][:, 2]
                xmax = width - target['boxes'][:, 0]
                target['boxes'][:, 0] = xmin
                target['boxes'][:, 2] = xmax
        return image, target
    
#ComposeWithTarget different transformations while also having transformation to the bouding boxes 
class ComposeWithTarget:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            image, target = t(image, target)
        return image, target

#AcneSet this is a class that contains the data set 
class AcneSet(Dataset):
    # __init__ will initialzie the data set that are found in the directory a
    def __init__(self, root_dir, set_type='train', transform=None):
        self.root_dir = root_dir
        self.set_type = set_type
        self.transform = transform
        self.image_dir = os.path.join(root_dir, set_type)
        self.annotation_dir = os.path.join(root_dir, set_type)
        self.images = [f for f in os.listdir(self.image_dir) if f.endswith('.jpg')]

    # will return the number of images that we have 
    def __len__(self):
        return len(self.images)
    
    # will load the image and its corrisponding bounding boxes 
    # it will extract the bouding boxes and the class labels 
    def __getitem__(self, idx):
        # constructs the full path of the image including bounding file 
        img_filename = self.images[idx]
        img_path = os.path.join(self.image_dir, img_filename)
        annotation_path = os.path.join(self.annotation_dir, img_filename.replace('.jpg', '.xml'))
        # loading the data and convert it to RBG if it is not already in it using convert('RBG')
        image = Image.open(img_path).convert('RGB')
        # pase the annotation boxes that are associated with the images using ET.parse to extract the bounding boxes 
        tree = ET.parse(annotation_path)
        root = tree.getroot()
        boxes = []
        labels = []
        # iterate over the object tag that is found in the tree ( the bounding boxes) in order to extract them and gather data about them and the class labels 
        for member in root.findall('object'):
            # retrive the name of the class 
            class_name = member.find('name').text 
            # map the class name to the integer ID using name_to_id dictrionay that was defined at the start of the code 
            class_id = name_to_id.get(class_name)  
            if class_id is not None: 
                # if the class name exists append class id to the labels and extract the bounding boxes 
                labels.append(class_id)
                # create a list of the bounding boxes for all the detected images 
                bbox = member.find('bndbox')
                xmin = float(bbox.find('xmin').text)
                ymin = float(bbox.find('ymin').text)
                xmax = float(bbox.find('xmax').text)
                ymax = float(bbox.find('ymax').text)
                boxes.append([xmin, ymin, xmax, ymax])
        #convert both lists the list of boxes and the list of labels to pytorch using torch.as_tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32) if boxes else torch.zeros((0, 4), dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        # create a target dictionary that contains the boxes and the class labels and the image id 
        target = {'boxes': boxes, 'labels': labels, 'image_id': torch.tensor([idx])}
        if self.transform:
            image, target = self.transform(image, target)
        return image, target


    '''
    get_transform is a static method thus it belongs to the class itself 
    it returns the ComposeWithTarget  which is a list that is composed of the data augmentation transformation that has been applied to the image and their annotation boxes 
    its arugment is train which is a boolean which indicates if those transformations are done for training if true and if it is false then it is for testing and validating 
    it 
    '''
    @staticmethod
    def get_transform(train=True):
        transforms_list = [
            ToTensorWithTarget(),
            ColorJitterWithTarget(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
            RandomHorizontalFlipWithBBox(p=0.5)
        ]
        return ComposeWithTarget(transforms_list)

#this is a dictionary that maps the class name  or the classification of the acne to the ID 
class_to_int = {
    'Acne': 1,
    'Blackhead': 2,
    'Conglobata': 3,
    'Crystanlline': 4,
    'Cystic': 5,
    'Flat_wart': 6,
    'Folliculitis': 7,
    'Keloid': 8,
    'Milium': 9,
    'Papular': 10,
    'Purulent': 11,
    'Scars': 12,
    'Sebo-crystan-conglo': 13,
    'Syringoma': 14,
    'Whitehead': 15,
    'null': 16
}

# this is a dictionary that contains the count of each class that is in our data set 
class_counts = {
    'Acne': 5314,
    'Blackhead': 799,
    'Conglobata': 8,
    'Crystanlline': 62,
    'Cystic': 438,
    'Flat_wart': 240,
    'Folliculitis': 237,
    'Keloid': 255,
    'Milium': 330,
    'Papular': 1457,
    'Purulent': 1183,
    'Scars': 388,
    'Sebo-crystan-conglo': 548,
    'Syringoma': 115,
    'Whitehead': 775,
    'null': 50
}


#it is also a dictionary that calculates the class weight based on the inverse of the class count which is a dictionary that is defined earlier
class_weights = {class_to_int[cls]: 1.0 / count for cls, count in class_counts.items()}

train_dataset = AcneSet(root_dir='dataset', set_type='train', transform=AcneSet.get_transform(train=True))

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'dataset\\train'

In [ ]:
'''
this code will handle the weights for each sample in the data set for training 
'''

# it will first check if samples are found in the folder
weights_file = 'sample_weights.pt'
# if the samples dont exist then it will calculate the weights for each sample in the data set for the training based on the class weights and it will also save them to a file 
if not os.path.exists(weights_file):
    print("weights dont exist, generating")
    total_samples = len(train_dataset)
    sample_weights = []
    '''
    for each weight it will calculate it based on the class weight with the label of each image 
    if the label doesnt have a weight in the class weight then it sets it 0 
    '''
    for idx in range(len(train_dataset)):
        _, target = train_dataset[idx]
        label_weights = [class_weights.get(label.item(), 0) for label in target['labels']]
        weight = sum(label_weights) / len(label_weights) if label_weights else 0
        sample_weights.append(weight)
        sys.stdout.write(f"\rProcessed sample {idx+1}/{total_samples}")
        sys.stdout.flush()

    sample_weights = torch.tensor(sample_weights)
    # it will save the weights that have been calculates to the file sample_weights.pt using the torch.save
    torch.save(sample_weights, weights_file)
    print("done saving weights to file")
else:
    print("weights exist already, loading them")
    sample_weights = torch.load(weights_file)
'''
we created weighted sampler using the same weight it is used to sample fromt the dataset during the trianing it will also set replacement to true , 
the number of samples equals to the number of samples that is found in the data set 
'''
weighted_sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

weights exist already, loading them


In [ ]:
import matplotlib.pyplot as plt
'''
train_epoch this function trains the model for each epoch 
using the data loader and optimizer 
it will iterate over the batches of the data 
it will move those batches to a specific device 
it will compute the loss 
it will performs backpropagation
it will update the models parameter 
it returns the average training loss for every epoch it trained on 
'''
def train_epoch(model, train_loader, optimizer, device, epoch, num_epochs):
    # sets the model to training mode 
    model.train()
    # initalizing variabales to keep track of the total loss and the total batches in the training laoder
    total_loss = 0
    total_batches = len(train_loader)
    #it will iterate over each batch using enumerate(trainloder)
    for batch_idx, (images, targets) in enumerate(train_loader):
        # images and targets are moves to a specific device
        images = [image.to(device) for image in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        # clear the gards of the optimizer 
        optimizer.zero_grad()
        # compute the loss for each batch in the trainin g
        loss_dict = model(images, targets)
        # calculates the total loss by summing the individual loss using sum
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        # update the model optimizer 
        optimizer.step()
        total_loss += losses.item()
        # printing the batches 
        print(f"Epoch {epoch+1}/{num_epochs} - Batch {batch_idx+1}/{total_batches}, Loss: {losses.item():.4f}")
    avg_loss = total_loss / total_batches
    print(f"Average training loss: {avg_loss:.4f}")
    return avg_loss


'''
calculate the IoU whichis the intersection over the union betweent he predicted boxes and the ground to truth boxes 
input: boxes-preds and boxes-gt which are arrays each boxes represents the bounding boxes in the format of [x-min,y-min,x-max,x-min]
'''
def calculate_iou_vectorized(boxes_preds, boxes_gt):
    # xA y A xB yB represent the corrdinates of the intersection between the two boxes
    xA = np.maximum(boxes_preds[:, None, 0], boxes_gt[None, :, 0])
    yA = np.maximum(boxes_preds[:, None, 1], boxes_gt[None, :, 1])
    xB = np.minimum(boxes_preds[:, None, 2], boxes_gt[None, :, 2])
    yB = np.minimum(boxes_preds[:, None, 3], boxes_gt[None, :, 3])
    # calculate the area of the intersection between the two boxes
    interArea = np.maximum(0, xB - xA + 1) * np.maximum(0, yB - yA + 1)
    # calculate the area of each box alone
    boxAArea = (boxes_preds[:, 2] - boxes_preds[:, 0] + 1) * (boxes_preds[:, 3] - boxes_preds[:, 1] + 1)
    boxBArea = (boxes_gt[:, 2] - boxes_gt[:, 0] + 1) * (boxes_gt[:, 3] - boxes_gt[:, 1] + 1)
    # calculate the IoU for each box and return it as a numpy array 
    iou = interArea / (boxAArea[:, None] + boxBArea - interArea)

    return iou
'''
get_model_predictions will generate predicitons using a trained object detection model 
'''
def get_model_predictions(model, data_loader, device):
    # sets the model to evaluation mode
    model.eval()
    # initalizes the array to store the predicitons that will be generated
    all_predictions = []
    with torch.no_grad():
        # it will iterate over the batches that are in the data loader 
        for images, targets in tqdm(data_loader, desc="Predicting", leave=False):
            # it will move the images to a specific device and compute the predicition using the model
            images = list(img.to(device) for img in images)
            outputs = model(images)
            # the predicited  boxes and scores and labels are extracted and converted to a comment format 
            for output in outputs:
                boxes = output['boxes'].data.cpu().numpy()
                scores = output['scores'].data.cpu().numpy()
                labels = output['labels'].data.cpu().numpy()
                # the predicitions are stores in a form of dictionary 
                for box, score, label in zip(boxes, scores, labels):
                    bbox = [box[0], box[1], box[2] - box[0], box[3] - box[1]]
                    all_predictions.append({'bbox': bbox, 'score': score, 'label': label})
    # a log message to indicate that the predicitions were successfully generated 
    logging.info("Predictions generated.")
    return all_predictions
'''
get_ground_truths the function will extract the ground truth and labels using the data loader 
'''
def get_ground_truths(data_loader, category_id_mapping):
    # initalize an array for the groun truth
    ground_truths = []
    # iterate through the batches of the data from the data loader
    # extract the images, boxes, labels
    for images, targets in tqdm(data_loader, desc="Extracting Ground Truths", leave=False):
        for target in targets:
            # bounding boxes are converted to the format [x-min,x-max,y-min,y-max]
            image_id = target["image_id"].item()
            boxes = target['boxes'].data.cpu().numpy()
            labels = target['labels'].data.cpu().numpy()
            for box, label in zip(boxes, labels):
                # the information is then stored in the form of dictionary 
                bbox = [box[0], box[1], box[2] - box[0], box[3] - box[1]]
                # the keys of the dictionary are image_id, catergory_id, bbox
                ground_truths.append({
                    'image_id': image_id,
                    'category_id': category_id_mapping[label.item()],
                    'bbox': bbox
                })
    # after processing all the batches a log message is printed indicating sucess  
    logging.info("Ground truths extraction completed.")
    return ground_truths
'''
calculate_metrics_vectorized calculate the precision and the f1 score 
'''
def calculate_metrics_vectorized(predictions, ground_truths, iou_threshold=0.5):
    # extracting the boxes of predicition and the ground to truth 
    boxes_preds = np.array([p['bbox'] for p in predictions])
    boxes_gt = np.array([gt['bbox'] for gt in ground_truths])
    labels_gt = np.array([gt['category_id'] for gt in ground_truths])  
    # calculate the IoU by calling the previously defined calculate_iou_vectorized function
    iou_matrix = calculate_iou_vectorized(boxes_preds, boxes_gt)
    # a logical matrix 'matches' is created 
    matches = (iou_matrix >= iou_threshold)
    # true positive is calculated by summing the matches per ground truth label
    tp = np.sum(matches.any(axis=0))
    # false positive is calculates by summing the matches per predicition label where no matches with ground truth exists 
    fp = np.sum(~matches.any(axis=1))
    # False negatives calculated by subtracting true positive from the total number of the ground truth 
    fn = len(ground_truths) - tp
    # calculating precision and recall and the F1 score 
    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1_score
'''
simplified_map computed the mean average precisions
'''
def simplified_map(predictions, ground_truths, iou_thresholds=[0.5]):
    # initalizing an array to store the output
    average_precisions = []
    #iterating over each threshold 
    for iou_threshold in iou_thresholds:
        # calculates the precision using a predefined function calculate_metrics_vectorized
        precision, _, _ = calculate_metrics_vectorized(predictions, ground_truths, iou_threshold=iou_threshold)
        # append the result to the average precision array 
        average_precisions.append(precision)
    # compute the mean average and return it 
    return sum(average_precisions) / len(average_precisions)
'''
validate this function will perform validation using precision and recall and F1 score and the mAP
'''
def validate(model, data_loader, device, category_id_mapping):
    print("Starting validation...")
    # sets the model to evaluation mode
    model.eval()
    with torch.no_grad():
        # it will generate a predicition and ground truth using predefined functions get_model_predictions get_ground_truths
        predictions = get_model_predictions(model, data_loader, device)
        ground_truths = get_ground_truths(data_loader, category_id_mapping)
        # it will then calculate the precision and the recall and the f1 score using a predefined function calculate_metrics_vectorized
        precision, recall, f1_score = calculate_metrics_vectorized(predictions, ground_truths, iou_threshold=0.5)
        # it will then calculate mAP using a pre defined funciton simplified_map
        mAP = simplified_map(predictions, ground_truths, iou_thresholds=[0.5])
        print(f"Validation completed. Precision: {precision * 100:.2f}%, Recall: {recall * 100:.2f}%, F1 Score: {f1_score * 100:.2f}%, mAP: {mAP:.2f}")
        return precision, recall, f1_score, mAP

'''
this function will plot the trianing loss, validation precisssion and the validation recall , f1 score and the validation mAP over each epoch
'''
def plot_metrics(train_losses, val_precisions, val_recalls, val_f1_scores, val_maps):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(20, 4))

    # creating a subplot for every matix 
    plt.subplot(1, 5, 1)
    plt.plot(epochs, train_losses, 'b-o', label='Train Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.legend()
    # Function to filter out None values and multiply by 100
    def prepare_for_plotting(metrics):
        return [m * 100 for m in metrics if m is not None]

'''
This function plots the training loss over epochs in a separate plot.
'''
def plot_training_loss(train_losses):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(10, 4))
    plt.plot(epochs, train_losses, 'b-o', label='Train Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training Loss Over Epochs')
    plt.legend()
    plt.show()

'''
his function plots validation precision, recall, F1-score, and mAP over epochs in subplots. 
It supports plotting multiple metrics simultaneously and determines the layout of subplots based on the number of metrics to plot.
'''
def plot_metrics(val_precisions, val_recalls, val_f1_scores, val_maps):
    valid_epochs = range(1, len(val_precisions) + 1)
    metrics_to_plot = [val_precisions, val_recalls, val_f1_scores, val_maps]
    metric_labels = ['Precision (%)', 'Recall (%)', 'F1 Score (%)', 'mAP (%)']
    metric_colors = ['orange', 'green', 'red', 'purple']
    # Function to filter out None values and multiply by 100
    def prepare_for_plotting(metrics):
        return [m * 100 for m in metrics if m is not None]
    # Determine the number of rows needed for subplots
    num_metrics = len(metrics_to_plot)
    num_rows = (num_metrics + 1) // 2
    plt.figure(figsize=(20, 4 * num_rows))
    for i, (metrics, label, color) in enumerate(zip(metrics_to_plot, metric_labels, metric_colors), start=1):
        plt.subplot(num_rows, 2, i)
        plt.plot(valid_epochs, prepare_for_plotting(metrics), color=color, marker='o', linestyle='-', label=f'Validation {label}')
        plt.xlabel('Epochs')
        plt.ylabel(label)
        plt.title(f'Validation {label} Over Epochs')
        plt.legend()
    plt.tight_layout()
    plt.show()




"\ndef plot_metrics(train_losses, val_precisions, val_recalls, val_f1_scores, val_maps):\n    epochs = range(1, len(train_losses) + 1) \n    plt.figure(figsize=(20, 4))\n\n    # Plotting training loss\n    plt.subplot(1, 5, 1)\n    plt.plot(epochs, train_losses, 'b-o', label='Train Loss')\n    plt.xlabel('Epochs')\n    plt.ylabel('Loss')\n    plt.title('Training Loss')\n    plt.legend()\n\n    # Plotting precision\n    plt.subplot(1, 5, 2)\n    plt.plot(epochs, val_precisions, 'orange-o', label='Validation Precision')\n    plt.xlabel('Epochs')\n    plt.ylabel('Precision')\n    plt.title('Validation Precision')\n    plt.legend()\n\n    # Plotting recall\n    plt.subplot(1, 5, 3)\n    plt.plot(epochs, val_recalls, 'g-o', label='Validation Recall')\n    plt.xlabel('Epochs')\n    plt.ylabel('Recall')\n    plt.title('Validation Recall')\n    plt.legend()\n\n    # Plotting F1 Score\n    plt.subplot(1, 5, 4)\n    plt.plot(epochs, val_f1_scores, 'r-o', label='Validation F1 Score')\n    plt.xla

In [ ]:
from torch.utils.data.sampler import SubsetRandomSampler

'''
it will retrive a pre trained model FASTER-RCNN with RESNET 50 as a backbone and feature pyramid network 
'''
def get_model(num_classes):
    model_weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
    model = detection.fasterrcnn_resnet50_fpn(weights=model_weights)
    # Replace the classifier with a new one for the number of classes there are
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model


# a background class to represent areas that have no acne which is done by incrementing the number of classes by 1
num_classes = len(category_id_mapping) + 1
model = get_model(num_classes)
# CUDA setup and model initialization and check if they are available 
if torch.cuda.is_available():
    print("CUDA GPU is available.")
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA GPU is not available.")
#move the model to a specific device 
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)

# initalize the optimizer with a specific learning rate based on the number of epoches
lr = 0.005
momentum = 0.9
weight_decay = 0.0005
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
# setting up the learning rate scheduler in order to adjust the learning rate based on every epoch 
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)
# Initialize Dataset and DataLoader with a specific batch size and number of workers for data loading
batchSize = 32
numWorkers = 16
# sampling a a fixed subset of the dataset for every epoch that we have 
subset_size = len(train_dataset) // 22
subset_sampler = SubsetRandomSampler(range(subset_size))
train_loader = DataLoader(train_dataset, batch_size=batchSize, sampler=subset_sampler, collate_fn=collate_fn, num_workers=numWorkers)
valid_dataset = AcneSet(root_dir='dataset', set_type='valid', transform=AcneSet.get_transform(train=False))
valid_loader = DataLoader(valid_dataset, batch_size=batchSize, sampler=subset_sampler, collate_fn=collate_fn, num_workers=numWorkers)
# Initialize metric lists
train_losses = []
val_precisions = []
val_recalls = []
val_f1_scores = []
val_maps = []
#Training and Validating Phases
num_epochs = 5
# iterate over each epoch 
for epoch in range(num_epochs):
    # calling train_epoch whcih is a predefined function to train the model 
    train_loss = train_epoch(model, train_loader, optimizer, device, epoch, num_epochs)
    train_losses.append(train_loss)
    # validating over every 3 epoches 
    if (epoch + 1) % 3 == 0:  
        precision, recall, f1_score, mAP = validate(model, valid_loader, device, category_id_mapping)
        val_precisions.append(precision)
        val_recalls.append(recall)
        val_f1_scores.append(f1_score)
        val_maps.append(mAP)
    else:
        # appending the training loss and the validations to their list
        val_precisions.append(val_precisions[-1] if val_precisions else None)
        val_recalls.append(val_recalls[-1] if val_recalls else None)
        val_f1_scores.append(val_f1_scores[-1] if val_f1_scores else None)
        val_maps.append(val_maps[-1] if val_maps else None)
    
    print(f"Epoch {epoch+1}: Loss = {train_loss:.4f}")
# using a predefined function plot_metrics we are plotting our outcome 
plot_metrics(train_losses, val_precisions, val_recalls, val_f1_scores, val_maps)



CUDA GPU is available.
Device 0: NVIDIA GeForce RTX 4090
Epoch 1 of 5
Epoch 1/5 - Batch 1/9, Loss: 4.8999
Epoch 1/5 - Batch 2/9, Loss: 2.4255
Epoch 1/5 - Batch 3/9, Loss: 1.4330
Epoch 1/5 - Batch 4/9, Loss: 1.4100
Epoch 1/5 - Batch 5/9, Loss: 1.4694
Epoch 1/5 - Batch 6/9, Loss: 1.4992
Epoch 1/5 - Batch 7/9, Loss: 1.2057
Epoch 1/5 - Batch 8/9, Loss: 1.1525
Epoch 1/5 - Batch 9/9, Loss: 0.9216
Average training loss: 1.8241
Training Loss: 1.8241
Epoch 2 of 5
Epoch 2/5 - Batch 1/9, Loss: 1.0765
Epoch 2/5 - Batch 2/9, Loss: 1.2099
Epoch 2/5 - Batch 3/9, Loss: 1.1731
Epoch 2/5 - Batch 4/9, Loss: 0.9973
Epoch 2/5 - Batch 5/9, Loss: 1.4165
Epoch 2/5 - Batch 6/9, Loss: 1.2491
Epoch 2/5 - Batch 7/9, Loss: 1.2658
Epoch 2/5 - Batch 8/9, Loss: 1.2537
Epoch 2/5 - Batch 9/9, Loss: 1.2364
Average training loss: 1.2087
Training Loss: 1.2087
Epoch 3 of 5
Epoch 3/5 - Batch 1/9, Loss: 1.0831
Epoch 3/5 - Batch 2/9, Loss: 1.3121
Epoch 3/5 - Batch 3/9, Loss: 1.1852
Epoch 3/5 - Batch 4/9, Loss: 1.1105
Epoch 3/

Calculating Metrics:  40%|███▉      | 10540/26522 [03:42<05:43, 46.53it/s]